<a href="https://colab.research.google.com/github/appletreeleaf/Project/blob/main/%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C/Semantic_search_with_FAISS_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic search with FAISS (PyTorch)

```
transformer-based language model에서 text를
embedding vector로 표현합니다.
이렇게 구한 임베딩 벡터를 dot-product를 통해
corpus 내의 단어들과 유사도를 계산할 수 있죠.

이번 챕터에서는 임베딩 벡터를 활용한 emantic search engine을
구현해보겠습니다.
```

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

## Loading and preparing the dataset

In [ ]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset

In [ ]:
issues_dataset

In [ ]:
issuse_dataset = issues_dataset.filter(lambda x : (x['is_pull_request'] == False and len(x['comments']) > 0))
issuse_dataset

In [ ]:
columns = issuse_dataset.column_names
columns_to_keep = ['title', 'body', 'html_url', 'comments']
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issuse_dataset = issuse_dataset.remove_columns(columns_to_remove)
issuse_dataset

In [ ]:
issuse_dataset.set_format("pandas")
df = issuse_dataset[:]

```
dataset 형태에서 comments를 한 번 필터링해줬음에도
여전히 빈칸이 남아있다..
우선 한번 더 필터해주겠습니다.
```

In [ ]:
comments_df = df.explode("comments", ignore_index=True)
comments_df.head(4)

In [ ]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset[1]

```
comments 칼럼에 존재하는 comment들은
여러 sequence들이 하나의 리스트로 묶여있습니다.
이를 explode함수를 사용해 분할하여 늘어난 row에도
html_url, title, comments들을 똑같이 복사해주겠습니다.
```

-------------------------------------------

### ✏️ Try it out!
- See if you can use Dataset.map() to explode the comments column of issues_dataset without resorting to the use of Pandas. This is a little tricky; you might find the “Batch mapping” section of the 🤗 Datasets documentation useful for this task.

In [ ]:
tmp_dataset = issuse_dataset.map(lambda batch: {"new_comments": batch['comments']}, remove_columns=["comments"], batched=True)  # new column with 6 elements: [0, 1, 2, 0, 1, 2]
tmp_dataset

-------------------------------------------

In [ ]:
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)

```
comment들의 길이 정보를 담고있는
"comment_length" 칼럼을 추가해주겠습니다.
```

In [ ]:
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
comments_dataset

In [ ]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)
comments_dataset

```
마지막으로 title, body, comments를 concat
'text' 칼럼을 추가해주겠습니다.
```

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
import torch

device = torch.device("cuda")
model.to(device)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

In [ ]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

```
FAISS 알고리즘으로 'index'하기 위해
임베딩의 format을 array로 변경해줍니다.
```

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings")

In [ ]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
attached_indexes = embeddings_dataset.list_indexes()

for index in attached_indexes:
    embeddings_dataset.drop_index(index)
embeddings_dataset

In [ ]:
type(embeddings_dataset)

In [ ]:
embeddings_dataset.push_to_hub("appletreeleaf/refined-github-issues")